In [1]:
# ! pip install beautifulsoup4

https://kenpom.com/

In [2]:
import os
import sys
import pandas as pd
from bs4 import BeautifulSoup

In [31]:
def html2csv(year):
    path = f'{year}.html'
    # empty list
    data = []
    
    # for getting the header from
    # the HTML file
    list_header = []
    soup = BeautifulSoup(open(path),'html.parser')
    header = soup.find_all("table")[0].find("tr")
    
    for items in header:
        try:
            list_header.append(items.get_text())
        except:
            continue
    
    # for getting the data 
    HTML_data = soup.find_all("table")[0].find_all("tr")[1:]
    
    for element in HTML_data:
        sub_data = []
        for sub_element in element:
            try:
                sub_data.append(sub_element.get_text())
            except:
                continue
        data.append(sub_data)
    
    columns = [
        'RK',
        'TEAM',
        'CONF',
        'W-L', 
        'ADJEM', 
        'ADJOE', 'ADJOE_R', 
        'ADJDE', 'ADJDE_R',
        'ADJ_T', 'ADJ_T_R',
        'Luck', 'Luck_R',
        'SOS', 'SOS_R',
        'OppO', 'OppO_R',
        'OppD', 'OppD_R',
        'NCSOS', 'NCSOS_R'
    ]
    int_cols = [
        'RK',
        'ADJOE_R', 
        'ADJDE_R',
        'ADJ_T_R',
        'Luck_R',
        'SOS_R',
        'OppO_R',
        'OppD_R',
        'NCSOS_R'
    ]
    float_cols = [
        'ADJEM', 
        'ADJOE', 
        'ADJDE', 
        'ADJ_T', 
        'Luck', 
        'SOS', 
        'OppO', 
        'OppD', 
        'NCSOS', 
    ]
    
    for i in range(len(data)):
        data[i] = [x for x in data[i] if x != "\n" and x != ""]
        data[i] = [x if '*' not in x else ''.join(x.split(' ')[:-1]) for x in data[i]]
    data = [x for x in data if len(x) == 21]

    df = pd.DataFrame(data = data, columns = columns)
    def fix_ints(s):
        return ' '.join([x for x in ''.join([i for i in s if not i.isdigit()]).split(' ') if x != ''])
    df['TEAM'] = df['TEAM'].apply(fix_ints)

    df[int_cols] = df[int_cols].astype(int)
    df[float_cols] = df[float_cols].astype(float)
    df["YEAR"] = year
    def get_wins(s):
        return int(s.split('-')[0])
    def get_losses(s):
        return int(s.split('-')[1])
    def get_games(s):
        return int(s.split('-')[0]) + int(s.split('-')[1])
    df['W'] = df['W-L'].apply(get_wins)
    df['L'] = df['W-L'].apply(get_losses)
    df['G'] = df['W-L'].apply(get_games)
    return df

In [36]:
all_df = None
for i in range(23):
    df = html2csv(2002 + i)
    all_df = pd.concat((all_df, df), axis=0)
all_df

,RK,TEAM,CONF,W-L,ADJEM,ADJOE,ADJOE_R,ADJDE,ADJDE_R,ADJ_T,...,OppO,OppO_R,OppD,OppD_R,NCSOS,NCSOS_R,YEAR,W,L,G
0,1,Duke,ACC,31-4,34.19,121.0,1,86.8,1,74.5,...,109.1,13,99.2,31,6.66,34,2002,31,4,35
1,2,Cincinnati,CUSA,31-4,30.19,118.1,7,87.9,3,67.4,...,106.3,66,99.7,42,3.48,80,2002,31,4,35
2,3,Maryland,ACC,32-4,29.25,119.2,4,89.9,7,73.7,...,109.1,11,99.3,33,1.62,120,2002,32,4,36
3,4,Kansas,B12,33-4,28.99,118.7,5,89.7,6,77.3,...,110.3,4,99.6,40,8.32,23,2002,33,4,37
4,5,Oklahoma,B12,31-5,26.04,114.9,20,88.9,4,66.5,...,109.0,15,100.2,62,-0.45,169,2002,31,5,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,358,Houston Christian,Slnd,0-4,-21.19,95.7,331,116.9,360,75.8,...,105.8,119,103.0,130,2.77,121,2024,0,4,4
358,359,Saint Francis,NEC,2-5,-23.13,91.6,357,114.8,355,66.8,...,108.4,41,98.5,17,9.81,27,2024,2,5,7
359,360,North Carolina A&T,CAA,0-6,-23.16,96.3,322,119.5,362,72.9,...,105.8,120,99.8,38,5.91,64,2024,0,6,6
360,361,Coppin St.,MEAC,0-7,-25.51,85.4,362,110.9,322,65.7,...,104.1,206,105.1,210,-1.05,215,2024,0,7,7


In [41]:
all_df.to_csv('kenpom.csv', index = False)

In [7]:
df.dtypes

RK           int32
TEAM        object
CONF        object
W-L         object
ADJEM      float64
ADJOE      float64
ADJOE_R      int32
ADJDE      float64
ADJDE_R      int32
ADJ_T      float64
ADJ_T_R      int32
Luck       float64
Luck_R       int32
SOS        float64
SOS_R        int32
OppO       float64
OppO_R       int32
OppD       float64
OppD_R       int32
NCSOS      float64
NCSOS_R      int32
YEAR         int64
W            int64
L            int64
G            int64
dtype: object

In [9]:
df

,RK,TEAM,CONF,W-L,ADJEM,ADJOE,ADJOE_R,ADJDE,ADJDE_R,ADJ_T,...,OppO,OppO_R,OppD,OppD_R,NCSOS,NCSOS_R,YEAR,W,L,G
0,1,Kansas,B12,28-3,30.23,115.8,8,85.5,2,67.3,...,107.4,26,94.7,1,9.58,10,2020,28,3,31
1,2,Gonzaga,WCC,31-2,26.95,121.3,1,94.4,43,71.9,...,103.5,115,101.0,99,-2.09,245,2020,31,2,33
2,3,Baylor,B12,26-4,25.49,113.5,17,88.1,4,66.2,...,106.4,39,96.2,11,1.38,132,2020,26,4,30
3,4,Dayton,A10,29-2,24.93,119.1,2,94.1,38,67.6,...,104.1,103,101.3,105,-0.74,203,2020,29,2,31
4,5,Duke,ACC,25-6,24.62,115.7,9,91.1,12,72.0,...,106.0,53,98.7,58,2.60,85,2020,25,6,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,349,Maryland Eastern Shore,MEAC,5-27,-26.14,81.7,352,107.8,288,67.5,...,97.6,337,104.1,226,7.78,15,2020,5,27,32
349,350,Howard,MEAC,4-29,-26.15,93.6,326,119.7,350,70.2,...,96.7,344,105.0,274,0.96,152,2020,4,29,33
350,351,Mississippi Valley St.,SWAC,4-27,-27.87,89.8,345,117.6,349,77.1,...,97.8,334,103.9,215,5.14,37,2020,4,27,31
351,352,Kennesaw St.,ASun,1-28,-28.71,83.5,351,112.2,329,68.2,...,102.0,187,103.7,204,4.10,56,2020,1,28,29
